In [1]:
pip install selenium

     |████████████████████████████████| 9.7 MB 92 kB/s  eta 0:00:01
     |████████████████████████████████| 126 kB 6.7 MB/s eta 0:00:01
     |████████████████████████████████| 164 kB 12.1 MB/s eta 0:00:01
     |████████████████████████████████| 481 kB 20.6 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 891 kB/s eta 0:00:011
     |████████████████████████████████| 63 kB 429 kB/s  eta 0:00:01
     |████████████████████████████████| 58 kB 1.4 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

     |████████████████████████████████| 12.4 MB 6.8 MB/s eta 0:00:01 MB 3.6 MB/s eta 0:00:03
     |████████████████████████████████| 508 kB 12.5 MB/s eta 0:00:01
     |████████████████████████████████| 346 kB 5.5 MB/s eta 0:00:01
     |████████████████████████████████| 17.3 MB 19.0 MB/s eta 0:00:01    |███████████████████▎            | 10.4 MB 9.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pyodbc sqlalchemy


Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import random
import re

from sqlalchemy import create_engine, text, MetaData, Table, Column, Integer, String, ForeignKey
from sqlalchemy.orm import sessionmaker

## Inicializando uma instância do google chrome

In [3]:
# A classe service é usada para iniciar uma instancia do chrome webdriver
service = Service()

# webdriver.Chromeoptions é usado para definir a preferência para o brower do chrome
options = webdriver.ChromeOptions()

# Inicia-se a instância do chrome WebDriver com as definidas 'options' e 'service'
driver = webdriver.Chrome(service=service, options=options)

In [4]:
url = "https://www.espn.com.br/nba/classificacao/_/grupo/liga"

driver.get(url)

## Raspagem dos Times

In [5]:
# Encontra todos os elementos com a tag <a>
all_teams = []

selector = 'a[data-clubhouse-uid^="s:40~l:46~t:"]'
teams = driver.find_elements(By.CSS_SELECTOR, selector)

# Filtra os elementos para remover espaços em branco ou elementos sem texto
all_teams = [team for team in teams if team.text.strip()]

print(f"Encontrados {len(all_teams)} times")

for team in all_teams:
  print(team.text)

team_names = [team.text for team in all_teams]

links = [link.get_attribute('href') for link in all_teams]
print(links)


Encontrados 30 times
Cleveland Cavaliers
Oklahoma City Thunder
Boston Celtics
Houston Rockets
New York Knicks
Memphis Grizzlies
Denver Nuggets
Dallas Mavericks
Orlando Magic
Los Angeles Lakers
Milwaukee Bucks
Minnesota Timberwolves
LA Clippers
Miami Heat
Indiana Pacers
Golden State Warriors
Detroit Pistons
Atlanta Hawks
San Antonio Spurs
Sacramento Kings
Phoenix Suns
Chicago Bulls
Philadelphia 76ers
Brooklyn Nets
Portland Trail Blazers
Utah Jazz
Charlotte Hornets
Toronto Raptors
New Orleans Pelicans
Washington Wizards
['https://www.espn.com.br/nba/time/_/nome/cle/cleveland-cavaliers', 'https://www.espn.com.br/nba/time/_/nome/okc/oklahoma-city-thunder', 'https://www.espn.com.br/nba/time/_/nome/bos/boston-celtics', 'https://www.espn.com.br/nba/time/_/nome/hou/houston-rockets', 'https://www.espn.com.br/nba/time/_/nome/ny/new-york-knicks', 'https://www.espn.com.br/nba/time/_/nome/mem/memphis-grizzlies', 'https://www.espn.com.br/nba/time/_/nome/den/denver-nuggets', 'https://www.espn.com.br/

### Raspagem dos jogadores

In [8]:

list_teams = []

for i, link in enumerate(links):
  driver.get(links[i])
  selector = 'a[data-track-nav_item="elenco"]'
  driver.find_element(By.CSS_SELECTOR, selector).click()

  time.sleep(random.uniform(2, 5))
  team_data = {}
  list_players = []
  
  
  # Encontra a tabela (sem classes)
  table = driver.find_element(By.TAG_NAME, "table")

  # Encontra todas as linhas (tr) da tabela
  rows = table.find_elements(By.TAG_NAME, "tr")

  # Itera pelas linhas da tabela (ignorando o cabeçalho, se existir)
  for row in rows[1:]: # Ignora o cabeçalho (primeira linha)
    player_data = {}  
    cells = row.find_elements(By.TAG_NAME, "td")  # Encontra todas as células

    if cells:  # Verifica se a linha contém células (se não for uma linha vazia)
        # Pegue a imagem
        image_player = row.find_element(By.TAG_NAME, "img").get_attribute("alt") 

        # Extrai o texto das células da linha
        player_info = [cell.text.strip() for cell in cells]

        # Atribui os dados ao dicionário de jogador
        if len(player_info) >= 7:
            player_data["image"] = image_player
            player_data["nome"] = re.match(r"\D+",player_info[1]).group(0) if player_info[1] else None
            player_data["numero"] = re.search(r"\d+",player_info[1]).group(0) if re.search(r"\d+", player_info[1]) else None
            player_data["pos"] = player_info[2]
            player_data["idade"] = player_info[3]
            player_data["alt"] = player_info[4]
            player_data["p"] = player_info[5]
            player_data["univ"] = player_info[6]
            player_data["salario"] = player_info[7]

            # Adiciona o jogador à lista de jogadores
            list_players.append(player_data)

  technical = driver.find_element(By.CLASS_NAME, "TableDetails__Paragraph").text.replace("Nome do técnico", "").strip()
  image_link = driver.find_element(By.CLASS_NAME, "Logo__xxl").get_attribute('src')
    
  team_data = {
    "id": i,
    "time": team_names[i],
    "technical": technical,
    "image_link": image_link,
    "players": list_players,
  }

  list_teams.append(team_data)


### Inserindo os dados na Base de dados

In [27]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus

server = "localhost,1433"
user = "sa"
password = "SuperPassword@24"
db_name = "NBAStats"
dsn = "ODBC Driver 18 for SQL Server"

engine = create_engine(f"mssql+pyodbc://{user}:%s@{server}/{db_name}?TrustServerCertificate=yes&driver={dsn}" % quote_plus(password))

connection = engine.connect()



In [39]:

# Crie uma sessão para interagir com o banco
Session = sessionmaker(bind=engine)
session = Session()



for team in list_teams:
    team_id = team["id"]
    sql_query = text("""
    INSERT INTO dbo.team (id, team_name, image, technical) 
    VALUES (:id, :team_name, :image, :technical)
    """)

    session.execute(sql_query, {"id": team["id"], "team_name": team["time"], "image": team["image_link"], "technical": team["technical"]})

    for player in team["players"]:
        sql_query = text("""
            INSERT INTO player (player_name, player_number, pos, age, height, weight, college, salary, image_link, team_id)
            VALUES (:player_name, :player_number, :pos, :age, :height, :weight, :college, :salary, :image_link, :team_id)
            """)
        session.execute(sql_query,
            {
                "player_name": player["nome"],
                "player_number": player["numero"],
                "pos": player["pos"],
                "age": player["idade"],
                "height": player["alt"],
                "weight": player["p"],
                "college": player["univ"],
                "salary": player["salario"],
                "image_link": player["image"],
                "team_id": team_id  # Relacionando o jogador ao time correto
            }
        )
    
    session.commit()  # Commit após inserir todos os jogadores de um time
        

print("Time inserido com sucesso!")


Time inserido com sucesso!
